In [56]:
import boto3
import sagemaker

In [57]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [58]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### Define hyperparameter

In [59]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

In [60]:
instance_count = 1
# instance_type = "ml.m5.large"
instance_type = "local"
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [61]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

### Define training cluster

In [62]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir='src',
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.3-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    max_wait=max_wait,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: local.


### Prepare training dataset

In [63]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ./data/dataset/ $data_path

In [64]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + '/data/dataset/'
    
else:
    inputs = data_path
inputs

'file:///home/ec2-user/SageMaker/sg-workshop/data/dataset/'

### Define Experiment

In [65]:
!pip install -U sagemaker-experiments

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [66]:
experiment_name='xgb-1'

In [67]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from time import strftime

In [68]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)
        
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name        

### HPO

In [69]:
max_jobs=4    # TODO: Ideally 12 or more
max_parallel_jobs=2   # TODO: Maybe only 1 for Event Engine, 2-3 if possible

In [70]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

job_name =  job_name[15:] ## job_name must have length less than or equal to 32 for HPO

tuner = sagemaker.tuner.HyperparameterTuner(
    estimator,
    objective_metric_name="validation:auc",
    hyperparameter_ranges={
        "max_depth": sagemaker.tuner.IntegerParameter(2, 5),
        "eta": sagemaker.tuner.ContinuousParameter(0.1, 0.5)
    },
    objective_type="Maximize",
    max_jobs=max_jobs,    # TODO: Ideally 12 or more
    max_parallel_jobs=max_parallel_jobs,    # TODO: Maybe only 1 for Event Engine, 2-3 if possible
)

tuner.fit(
    job_name = job_name,
    inputs={'inputdata': inputs},
    experiment_config={
          'TrialName': job_name,
          'TrialComponentDisplayName': job_name,
    },
    wait=False
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: 1663408433


AttributeError: 'LocalSagemakerClient' object has no attribute 'create_hyper_parameter_tuning_job'

In [38]:
tuner.wait()

NameError: name 'tuner' is not defined

### Result

In [39]:
from sagemaker.analytics import ExperimentAnalytics, HyperparameterTuningJobAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100

In [40]:
trial_component_training_analytics = HyperparameterTuningJobAnalytics(
    sagemaker_session= sagemaker_session,
    hyperparameter_tuning_job_name=job_name
)

trial_component_training_analytics.dataframe()[['TrainingJobName', 'TrainingJobStatus', 
                                                'eta', 'max_depth', 'FinalObjectiveValue']]

NameError: name 'job_name' is not defined

### Evaluation

In [41]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

In [42]:
instance_count = 1
#instance_type = "ml.m5.large"
instance_type = 'local'

In [43]:
script_eval = FrameworkProcessor(
    XGBoost,
    framework_version="1.3-1",
    role=role,
    instance_type=instance_type,
    instance_count=instance_count
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: local.


In [44]:
client = boto3.client('sagemaker')
response = client.describe_training_job(
    TrainingJobName=tuner.best_training_job()
)

NameError: name 'tuner' is not defined

In [53]:
artifacts_dir = response['ModelArtifacts']['S3ModelArtifacts']
artifacts_dir

NameError: name 'response' is not defined

In [54]:
s3_test_path = data_path + '/test.csv'
detect_outputpath = f's3://{bucket}/xgboost/processing'

In [46]:
from pathlib import Path
source_dir = f'{Path.cwd()}/src'

if instance_type == 'local':
    from sagemaker.local import LocalSession
    from pathlib import Path

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}

    s3_test_path=f'../data/dataset/test.csv'
else:
    sagemaker_session = sagemaker.session.Session()
    s3_test_path=data_path + '/test.csv'  

In [47]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

script_eval.run(
    code="evaluation.py",
    source_dir=source_dir,
    inputs=[ProcessingInput(source=s3_test_path, input_name="test_data", destination="/opt/ml/processing/test"),
            ProcessingInput(source=artifacts_dir, input_name="model_weight", destination="/opt/ml/processing/model")
    ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output", output_name='evaluation', destination=detect_outputpath + "/" + job_name),
    ],
    job_name=job_name,
    experiment_config={
        'TrialName': job_name,
        'TrialComponentDisplayName': job_name,
    },
    wait=False
)

NameError: name 'experiment_name' is not defined

In [48]:
script_eval.latest_job.wait()

AttributeError: 'NoneType' object has no attribute 'wait'

### Check Result

In [49]:
# artifacts_dir = xgb_estimator.model_data.replace('model.tar.gz', '')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

NameError: name 'artifacts_dir' is not defined

In [50]:
model_dir = './model'

!rm -rf $model_dir

import json , os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

!aws s3 cp {artifacts_dir} {model_dir}/model.tar.gz
!tar -xvzf {model_dir}/model.tar.gz -C {model_dir}


usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type
tar (child): ./model/model.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [51]:
!pip install xgboost graphviz

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [52]:
import xgboost as xgb
import matplotlib.pyplot as plt

test_prep_df = pd.read_csv('../data/dataset/test.csv')
x_test = test_prep_df.drop('fraud', axis=1)
feature_data = xgb.DMatrix(x_test)
model.get_booster().feature_names = feature_data.feature_names
model.get_booster().feature_types = feature_data.feature_types
fig, ax = plt.subplots(figsize=(15, 8))
xgb.plot_importance(model, ax=ax, importance_type='gain')

FileNotFoundError: [Errno 2] No such file or directory: '../data/dataset/test.csv'

In [ ]:
xgb.plot_tree(model, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(50, 15)
plt.show()